In [1]:
import re 
import os
import nltk 
from sklearn import feature_extraction
from gensim import models
import numpy as np
import pickle

In [2]:
with open('../modelos/docs_tokenizados_mauro_campbell.pkl', 'rb') as fp:
    docs_tokenizados = pickle.load(fp)

In [3]:
palavras = []
for doc in docs_tokenizados:
    palavras.extend(doc)
unicas = set(palavras)
len(unicas)

25037

In [4]:
caminho_modelo_w2v = "../modelos/w2v_mauro_campbell_1k_decisoes_tweet_tknzr_window_15_mincount_10.model"
if not os.path.isfile(caminho_modelo_w2v):
    raise FileExistsError("Modelo w2v nao encontrado.")

In [5]:
w2v_model = models.Word2Vec.load(caminho_modelo_w2v)
vocab = list(w2v_model.wv.vocab)
len(vocab)

6207

In [6]:
# Funcoes que retornam o indice da palavra no vocabulario criado pelo word2vec e vice-versa
def word2idx(word):
    return w2v_model.wv.vocab[word].index


def idx2word(idx):
    return w2v_model.wv.index2word[idx]

In [7]:
maxlen = 15
sentencas = []
next_tokens = []

for doc in docs_tokenizados:  # Cria as "sentencas" para cada documento do corpus
    for i in range(0, len(doc) - maxlen):
        
        lista_palavras = doc[i:i + maxlen]
        
        excluir_sentenca = False
        for palavra in lista_palavras:
            if palavra not in w2v_model.wv.vocab:
                excluir_sentenca = True
                break
                
        proxima_palavra = doc[i + maxlen]
        if proxima_palavra not in w2v_model.wv.vocab:
            excluir_sentenca = True
                
        if excluir_sentenca:
            continue
        
        sentencas.append(lista_palavras)
        next_tokens.append(proxima_palavra)

len(sentencas)

927472

In [8]:
# Criando os inputs(x) e targets(y)
x = np.zeros((len(sentencas), maxlen), dtype=np.int32)
y = np.zeros((len(sentencas)), dtype=np.int32)
for i, sentenca in enumerate(sentencas):
    for t, token in enumerate(sentenca):
        x[i, t] = word2idx(token)
    y[i] = word2idx(next_tokens[i])

In [9]:
dados_treino = [x,y]
with open('../modelos/dados_treino_mincount_10_mauro_campbell.pkl', 'wb') as fp:
    pickle.dump(dados_treino, fp)